In [9]:
import pandas as pd
import numpy as np
import re

from transformers import BertModel, BertTokenizer

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, SequentialSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

import matplotlib.pyplot as plt


RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [3]:
train_data = pd.read_csv("ckip_cat_train_pos.csv", encoding = "utf-8", converters={"name": lambda x: re.findall(r'[\u4e00-\u9fff]+',x)})
train_data = train_data[["name","product"]]
train = train_data

In [4]:
def to_string(name_list):
    s = ""
    for name in name_list:
        s += name
    return s

def labeling(product):
    products = product.split(";")
    return products

In [6]:
train["name"] = train["name"].map(to_string)
train["product"] = train["product"].map(labeling)

In [8]:
mlb = MultiLabelBinarizer()
x = train["name"].tolist()
questions = x
yt = mlb.fit_transform(train["product"])

In [10]:
# First Split for Train and Test
x_train,x_test,y_train,y_test = train_test_split(x, yt, test_size=0.1, random_state=RANDOM_SEED,shuffle=True)
# Next split Train in to training and validation
x_tr,x_val,y_tr,y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=RANDOM_SEED,shuffle=True)

In [11]:
class QTagDataset (Dataset):
    def __init__(self,quest,tags, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = quest
        self.labels = tags
        self.max_len = max_len
        
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, item_idx):
        text = self.text[item_idx]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True, # Add [CLS] [SEP]
            max_length= self.max_len,
            padding = 'max_length',
            return_token_type_ids= False,
            return_attention_mask= True, # Differentiates padded vs normal token
            truncation=True, # Truncate data beyond max length
            return_tensors = 'pt' # PyTorch Tensor format
          )
        
        input_ids = inputs['input_ids'].flatten()
        attn_mask = inputs['attention_mask'].flatten()
        #token_type_ids = inputs["token_type_ids"]
        
        return {
            'input_ids': input_ids ,
            'attention_mask': attn_mask,
            'label': torch.tensor(self.labels[item_idx], dtype=torch.float)
            
        }

NameError: name 'pl' is not defined